# Lagrange space on one-dimensional domains

In this activity, we become familiar with a computational object that can represent members of the simplest finite element space, called the Lagrange finite element space. These are *vector spaces of functions* on a spatial domain. We will begin by seeing how distinct finite elements on a mesh are glued together to form such a space. The glued function space is often called a *global finite element space* to contrast it with the spaces on each individual finite element, referred to as *local finite element spaces*.

We start by importing the Python objects we use in this notebook.

In [ ]:
import ngsolve as ng
from ngsolve.meshes import Make1DMesh
from ngsolve import H1, x, sin, Discontinuous, GridFunction
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import TABLEAU_COLORS as tc
colors = [tc[key] for key in tc]    

## One element

We create a mesh of the unit interval $[0, 1]$ consisting of a single finite element on the same interval, as follows.

In [ ]:
mesh1 = Make1DMesh(1) # one element mesh of [0, 1]
for p in mesh1.vertices:
    print(p.point)    # confirm the mesh point coordinates

In higher dimensions, the printed tuple `p.point` will show more coordinates. 

The quadratic Lagrange finite element on this single element is made in the `V` object below using the ngsolve facility `H1`. (The name `H1` refers to a Sobolev space related to the Lagrange space.) The `ndof` attribute stores the number of degrees of freedom of the element. In this case, we get the expected number of three degrees of freedom for the quadratic Lagrange element.  

In [ ]:
V1 = H1(mesh1, order=2) # set degree using the keyword "order"
V1.ndof

Functions of finite element spaces are represented by `GridFunction` objects in ngsolve.

In [ ]:
v = ng.GridFunction(V1)

If $\theta_i(x)$ denote the shape functions of this finite element, then any function in the finite element space takes the form 

$$
  v(x) = \sum_i v_i \theta_i(x)
$$

for some numbers $v_i$. The `GridFunction` object `v` represents this function $v$ and contains the numbers $v_i$ in its data attribute `v.vec`. By printing out `v.vec` we thus have complete information on the function, provided we know the shape functions $\theta_i$. Similarly, by setting the array `v.vec`, we can represent any function in the finite element space as we wish. 

Not all finite element codes use the same shape functions (even if the space they span is a  classical finite element space). Let us acquaint ourselves with the Lagrange shape functions $\theta_i(x)$ used by ngsolve. We need a small routine to evaluate the finite element function at many points (say a hundred, as set below) within an element and plot the result. 

In [ ]:
def plot1D(w, msh, style='k', title=None):  
  """
  Utility to plot 1D finite element functions
  """
  pts = [vtx.point[0] for vtx in msh.vertices]
  ev = [e.vertices for e in msh.Elements(ng.VOL)]
  for i in range(len(ev)):
    a = pts[ev[i][0].nr] 
    b = pts[ev[i][1].nr] 
    aa = min(a, b) + 0.01 * abs(b-a)
    bb = max(a, b) - 0.01 * abs(b-a)
    xs = np.linspace(aa, bb, num=100)
    ws = [w(msh(x)) for x in xs]
    plt.plot(xs, ws, style, alpha=0.5, lw=3)
  plt.xlabel('x'); plt.title(title)

In three distinct colors, we see the three shape functions of the quadratic Lagrange element used within ngsolve. We discover that the Lagrange shape functions in ngsolve are not exactly the shape functions generated by the degrees of freedom of the principal lattice. But note that (see exercises) the same finite element can admit many distinct unisolvent degrees of freedom.

In [ ]:
for i in range(3):
  v.vec[:] = 0
  v.vec[i] = 1
  plot1D(v, mesh1, style=colors[i], title='Three Lagrange shape functions $\\theta_i$')

## Gluing two elements 

In [ ]:
mesh2 = Make1DMesh(2)  # two element mesh
for p in mesh2.vertices:
    print(p.point)     # this mesh now has 3 points

On this mesh of two elements, let us start by examining two *independent* quadratic Lagrange elements, where   functions of one element are *not glued* to the next element. This is achieved using the `Discontinuous` facility in ngsolve. As expected we now find a space of dimension $3 + 3 = 6$.

In [ ]:
V2 = Discontinuous(H1(mesh2, order=2))
V2.ndof

In [ ]:
v = GridFunction(V2)
v.vec[:] = 0; v.vec[0] = 1
plot1D(v, mesh2, style='b', title='First discontinuous shape function')

In [ ]:
v.vec[:] = 0; v.vec[4] = 1
plot1D(v, mesh2, style='k:', title='Fourth discontinuous shape function')

If we glue these first and fourth functions together we get a continuous function. The global Lagrange finite element space accomplishes this by coalescing the first and fourth degrees of freedom into one, thereby merging the above two shape functions into one global shape function corresponding to the coalesced global degree of freedom. We obtain this global Lagrange finite element space in ngsolve by simply omitting the adjective `Discontinuous` from the prior space definition.

In [ ]:
V = H1(mesh2, order=2) # omitted "Discontinuous"
V.ndof

All *global shape functions* of this space can be visualized in the same way as before. We immediately see that none of them are discontinuous. 

In [ ]:
v = GridFunction(V)
c = ['b', 'r', 'g', ':y', ':m']
for i in range(V.ndof):
  v.vec[:] = 0 
  v.vec[i] = 1
  plot1D(v, mesh2, style=c[i], title='Continuous global shape functions')

The global finite element space consists of functions of the form 

$$
v(x) = \sum_i v_i \Theta_i(x)
$$

where $\Theta_i(x)$ are global shape functions. For the current example, we plotted them above. Note that when $\Theta_i$ is restricted to a single element, we obtain either the zero function or a local shape function $\theta_j$ of that element. A set of linear functionals $G_j$ on the global finite element space having the property $G_j(\Theta_j) = \delta_{ij}$ are called *global degrees of freedom.* Here $\delta_{ij}$ denotes the Kronecker delta (equals 1 if $i=j$ and 0 otherwise). 

## Coupling degrees of freedom

When some local degrees of freedom of distinct finite elements are fused together to form a single global degree of freedom, like we saw above, the result is called a *coupling degree of freedom*. In ngsolve, every global shape function, or equivalently every global degree of freedom, is given a `CouplingType`. When its `CouplingType` is `LOCAL_DOF`, it is *not* a coupling degree of freedom; all other values of `CouplingType` indicates a coupling degree of freedom. 

In [ ]:
for i in range(V.ndof):
    print(V.CouplingType(i))

In this case, we see that all degrees of freedom are coupling degrees of freedom except the last two. Each degree of freedom  marked `LOCAL_DOF` is identical to a local element degree of freedom within some element and its corresponding shape function is supported only on that single element. (You can verify that the last two shape functions in the current example are supported only on a single element by reviewing the plots of the global shape functions above.)

## Approximation

One method of creating a finite element approximation of some given function $f(x)$  is by the finite element interpolant. In ngsolve, another technique of creating approximations is offered as the default. This is in a method called `Set` that all `GridFunctions` have. It implements the *Oswald approximation,* obtained by first computing the $L_2$ projection of $f$ into each local finite element space and then assigning to every coupling degree of freedom the average of the degrees of freedom of the projection that were fused to form that coupling degree of freedom.

In [ ]:
v.Set(sin(5*x))
plot1D(sin(5*x), mesh2, style=':b')
plot1D(v, mesh2, 'k')

Clearly the approximation (solid curve) and the original (dotted sine curve) are close, but not identical. 

**Questions for discussion:**

- What happens when you lower the degree on the same mesh?

- What happens when you increase the degree on the same mesh?

- How can you tell that these approximations are not obtained by the canonical finite element interpolation?  

One way to improve the approximation is by increasing the degree. Another way is to make the mesh finer, as we see now. 

## The lowest order case

Partition the interval $[0, 1]$ using a uniform mesh $\{ x_i = i h: i=0, \ldots, N \}$ of $N+1$ points, where the mesh size is $h = 1/N$ and set  

$$
V_{h, 1} = \{ v: \; v \text{ is continuous and }
v|_{[x_i, x_{i+1}]} \text { is linear }\}.
$$

This vector space is called the **lowest order Lagrange finite element space** in the one-dimensional domain $[0,1]$. With $N$ as a variable to adjust, let us make  this space in ngsolve:

In [ ]:
N = 5 
mesh = Make1DMesh(N) 
V0 = H1(mesh, order=1)
V0.ndof

In [ ]:
v = GridFunction(V0)
v.Set(sin(5*x))
plot1D(sin(5*x), mesh, style=':b')
plot1D(v, mesh, 'k')

Does the approximation improve as $h \to 0$ or $N \to \infty$? 

In [ ]:
for i in range(N+1):
    v.vec[:] = 0
    v.vec[i] = 1
    plot1D(v, mesh, style=colors[i], title='Global shape functions $\\Theta_i$')

These functions $\Theta_i \in V_{h1}$ with the property 

$$
\Theta_i(x_j) = \delta_{ij}
$$

at every mesh point $x_j$ are often called **hat functions** colloquially. Note that although the statement $\Theta_i(x_j) = \delta_{ij}$ only gives the values of $\Theta_i$ at the mesh vertices $x_j$, that is enough to determine the function $\Theta_i$ everywhere, since it is linear in between the mesh points.

**Questions for discussion**

- What type of an object is `sin(x)`?
- Can you try other function expressions?

Please review [NGSolve i-Tutorial 1.2](https://ngsolve.org/docu/latest/i-tutorials/unit-1.2-coefficient/coefficientfunction.html) to learn more about `CoefficientFunction` objects.